## Spatial transcriptomics deconvolution visualization
We support to visualize the scatterpie chart for any deconvolution or label transferring tools.

Here we will describe how to use SPOTlight and Seurat Label transferring to visualize in stLearn

In [ ]:
import os
import sys
import numpy as np
import pandas as pd
import scanpy as sc
import stlearn as st

##### Set working directory for analysis

In [ ]:
cwd = '/media/bio/Disk/Research Data/EBV/omicverse'
os.chdir(cwd)
updated_dir = os.getcwd()
print("Updated working directory: ", updated_dir)

from pathlib import Path
saving_dir = Path('Results/10.NPC_ST_Analysis')
saving_dir.mkdir(parents=True, exist_ok=True)

## Loading the data

#### Reading in annotated AnnData object

In [ ]:
adata = sc.read_h5ad("Processed Data/GSE206245_NPC_ST_Cluster_Tangram.h5ad")
adata

In [ ]:
print(np.min(adata.X), np.max(adata.X))

In [ ]:
import pandas as pd

sample_response = {
    'NPC_ST05': 'CR',
    'NPC_ST06': 'CR',
    'NPC_ST07': 'PD',
    'NPC_ST08': 'CR',
    'NPC_ST09': 'PD',
    'NPC_ST10': 'PR',
    'NPC_ST11': 'CR',
    'NPC_ST12': 'CR',
    'NPC_ST16': 'PD',
    'NPC_ST17': 'SD',
    'NPC_ST18': 'PR',
    'NPC_ST19': 'PD'
}

response_map = pd.Series(sample_response, name='response')

adata.obs['response'] = adata.obs['sample_id'].map(response_map)

print(adata.obs[['sample_id', 'response']].head())
print(adata.obs['response'].value_counts(dropna=False))


In [ ]:
import omicverse as ov
from matplotlib import patheffects
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(4,4))
ov.pl.embedding(adata,
                  basis='X_umap',
                  color=['response'], 
                  palette='Paired',
                  show=False, legend_loc=None, add_outline=False, 
                  frameon='small',legend_fontoutline=2,ax=ax
                 )

ov.utils.gen_mpl_labels(
    adata,
    'response',
    exclude=("None",),  
    basis='X_umap',
    ax=ax,
    adjust_kwargs=dict(arrowprops=dict(arrowstyle='-', color='black')),
    text_kwargs=dict(fontsize= 9,weight='bold',
                     path_effects=[patheffects.withStroke(linewidth=2, foreground='w')] ),
)

plt.show()

In [ ]:
fig,ax=plt.subplots(figsize = (3,6))
ov.pl.cellstackarea(adata=adata,celltype_clusters='scNiche',
                    groupby='response',groupby_li=['CR','PR','SD','PD'],
                     legend=True,ax=ax,legend_awargs={})

# Save and Show plot
plt.savefig("Results/10.NPC_ST_Analysis/Cellstackarea_Response_scNiche.pdf", bbox_inches='tight')
plt.show()

In [ ]:
# import packages
import scimap as sm
import anndata as ad

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

os.makedirs("Processed Data/plots", exist_ok=True)

sample_ids = adata.obs['sample_id'].unique()
saved_paths = []

for sid in sample_ids:
    print(f"Processing {sid}...")

    adata_subset = adata[adata.obs['sample_id'] == sid].copy()
    adata_subset.obs['imagecol'] = adata_subset.obsm['spatial'][:, 0]
    adata_subset.obs['imagerow'] = adata_subset.obsm['spatial'][:, 1]

    if 'use_quality' not in adata_subset.uns['spatial'][sid]:
        adata_subset.uns['spatial'][sid]['use_quality'] = 'hires'

    pred_path = f"Processed Data/tangram_ct_pred_{sid}.csv"
    tangram_ct_pred = adata_subset.obsm['tangram_ct_pred'].T
    tangram_ct_pred.to_csv(pred_path)

    st.add.add_deconvolution(adata_subset, annotation_path=pred_path)

    output_folder = f"Processed Data/plots/deconv_{sid}"
    os.makedirs(output_folder, exist_ok=True)
    st.pl.deconvolution_plot(
        adata_subset,
        threshold=0.0,
        library_id=sid,
        spot_size=80,
        cmap="tab20",
        dpi=300,
        output=output_folder,
        show=False
    )
    saved_paths.append(f"{output_folder}/louvain.png")

n_cols = 4
n_rows = int(np.ceil(len(saved_paths) / n_cols))
fig, axs = plt.subplots(n_rows, n_cols, figsize=(n_cols * 5, n_rows * 5))

for i, path in enumerate(saved_paths):
    row, col = divmod(i, n_cols)
    ax = axs[row, col] if n_rows > 1 else axs[col]
    img = Image.open(path)
    ax.imshow(img)
    sid = os.path.basename(os.path.dirname(path)).replace("deconv_", "")
    ax.set_title(sid, fontsize=12)
    ax.axis('off')

for i in range(len(saved_paths), n_rows * n_cols):
    row, col = divmod(i, n_cols)
    ax = axs[row, col] if n_rows > 1 else axs[col]
    ax.axis('off')

plt.tight_layout()
plt.savefig("Processed Data/deconvolution_all_samples_singlepage.pdf", bbox_inches='tight', dpi=300)
plt.close()

print("✅ 所有样本图已拼图并保存为一个 PDF 页面：Processed Data/deconvolution_all_samples_singlepage.pdf")

In [ ]:

adata_subset = adata[adata.obs['sample_id'] == "NPC_ST05"].copy()
adata_subset.obs['imagecol'] = adata_subset.obsm['spatial'][:, 0]
adata_subset.obs['imagerow'] = adata_subset.obsm['spatial'][:, 1]
adata_subset.uns['spatial']["NPC_ST05"]['use_quality'] = 'hires'
tangram_ct_pred = adata_subset.obsm['tangram_ct_pred'].T
tangram_ct_pred.to_csv("Processed Data/tangram_ct_pred_stlearn.csv")
st.add.add_deconvolution(adata_subset, annotation_path="Processed Data/tangram_ct_pred_stlearn.csv")
st.pl.deconvolution_plot(adata_subset, threshold=0.0, library_id = "NPC_ST05", spot_size = 80, dpi = 300,
                         cmap="tab20",  # "tab10", "Paired", "Accent", "Set1"
                         )



**<span style="font-size:16px;">Session information：</span>**

In [ ]:
import sys
import platform
import pkg_resources

# Get Python version information
python_version = sys.version
# Get operating system information
os_info = platform.platform()
# Get system architecture information
architecture = platform.architecture()[0]
# Get CPU information
cpu_info = platform.processor()
# Print Session information
print("Python version:", python_version)
print("Operating system:", os_info)
print("System architecture:", architecture)
print("CPU info:", cpu_info)

# Print imported packages and their versions
print("\nImported packages and their versions:")
for package in pkg_resources.working_set:
    print(package.key, package.version)